In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pickle
from time import time

In [3]:
df = pd.read_csv('app/data/post_counts.csv')

In [4]:
df.head(10)

,subreddit,day,total
0,CircleofTrust,92,192789
1,CircleofTrust,93,139672
2,thanosdidnothingwrong,190,119575
3,CircleofTrust,94,55236
4,thanosdidnothingwrong,189,41484
5,CircleofTrust,95,30424
6,movietvreview,272,23004
7,inthesoulstone,190,20621
8,u_Head_Evidence,264,17634
9,u_TallPrune,264,17606


In [5]:
d = {}

In [ ]:
%%time
for i, row in df.iterrows():
    s = row.subreddit
    if s not in d:
        d[s] = [0]*366
    d[s][row.day-1] = row.total